In [1]:
import os
import argparse
import inference

2022-08-10 16:26:10.391597: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
class namespace():
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
args = namespace()
args.input_dir = "./test_paragraph_topic.json"
args.output_dir = "./output"
args.model_name_or_path = "./model/model.pt"
args.num_labels = len(inference.doc_topics)
args.batch_size = 256
args.seed = 42
args.generate = True

inference.seed_everything(args.seed)
inference.inference(args)

2022-08-10 16:26:12,279 INFO     Create model.........


404 Client Error: Not Found for url: https://huggingface.co/./model/model.pt/resolve/main/config.json


OSError: Can't load config for './model/model.pt'. Make sure that:

- './model/model.pt' is a correct model identifier listed on 'https://huggingface.co/models'

- or './model/model.pt' is the correct path to a directory containing a config.json file

